In [1]:
#Pytorch
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
#Python Libs
import numpy as np
from time import time
import datetime
#Implementations
from model.agree_re import NCF
from config_douban import Config
from dataset_Meetup import GDataset
from batch_test import *

In [2]:
def training(model, train_loader, epoch_id, config):
    # user trainning
    t1 = time()
    model.train()
    learning_rates = config.lr
    lr = learning_rates[0]

    # optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    losses = []

    *_, last = train_loader

    for batch_id, (g, pi_ni) in enumerate(train_loader):
        # Data Load
        group_input = g
        pos_item_input = pi_ni[:, 0]
        neg_item_input = pi_ni[:, 1]

        optimizer.zero_grad()
        model.zero_grad()

        pos_prediction = model(group_input, pos_item_input)
        neg_prediction = model(group_input, neg_item_input)
        #print(pos_prediction[0], neg_prediction[0])

        # Loss

        loss = torch.mean((pos_prediction - neg_prediction -1) **2)
        
        # record loss history
        #print("batch_id: " + str(batch_id) + " loss: " + str(loss.item()))
        if not torch.isinf(loss.data) and not torch.isnan(loss.data):
            losses.append(float(loss.item()))
        # Backward
        loss.backward()
        optimizer.step()

    print('Iteration %d,\tloss: [%.4f], time: [%.1fs]' % (epoch_id, np.mean(np.array(losses)), time() - t1))


In [3]:
def evaluation(model, groups_to_test, Ks, trainRatings, num_items):
    model.eval()
    t2 = time()
    ret = test(model, groups_to_test, Ks, trainRatings, num_items) #See batch_test

    print('\t Evaluation done [%.1f s]' % (time() - t2))
    for i, k in enumerate(Ks):
        print('\t\t @%d: HR = %.4f, NDCG = %.4f, Rec = %.4f' % (k, ret['hit_ratio'][i], ret['ndcg'][i], ret['recall'][i]))
    return ret

In [4]:
config = Config()

douban 100 256 [0.0005, 1e-06, 5e-07]


In [5]:
dataset = GDataset(config.user_dataset, config.group_dataset, config.user_in_group_path, 1)
num_groups, num_users, num_items = dataset.num_groups, dataset.num_users, dataset.num_items
print("num_groups: "+str(num_groups)+" num_users: "+str(num_users)+" num_items: "+str(num_items))
gu_dict = dataset.gu_dict

num_groups: 92489 num_users: 63893 num_items: 15741


In [6]:
test_warm = dataset.load_rating_file_as_dict(config.group_dataset + ".test.rating_warm")
test_cold = dataset.load_rating_file_as_dict(config.group_dataset + ".test.rating_cold")
valid = dataset.load_rating_file_as_dict(config.group_dataset + ".valid.rating")

In [7]:
agree = NCF(num_users, num_items, num_groups, config.embedding_size, gu_dict, 0.5)#.cuda()
best_checkpoint = -1.0
best_weights_path = None
for num_negatives in config.num_negatives:
    dataset.num_negatives = num_negatives
    print("NCF: embedding size %d, run Iteration: %d, #neg: %d" %(config.embedding_size, config.epoch, num_negatives))
    # train the model
    now = datetime.datetime.now().strftime("%Y-%m-%d")

    for epoch in range(config.epoch):
        training(agree, dataset.get_group_dataloader(config.batch_size), epoch, config)
        

        # Evaluation
        if epoch % 3 == 0:
            agree.eval()
            with torch.no_grad():
                #ret = evaluation(agree, test_warm, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
                ret = evaluation(agree, valid, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
                cur_checkpoint = ret['hit_ratio'][1]
                #ret = evaluation(agree, test_cold, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
                #cur_checkpoint += ret['hit_ratio'][1]
                #cur_checkpoint = cur_checkpoint/2
                current_weights_path = 'weights/ncf'+str(config.dataset)+"_"+str(config.embedding_size)+"_"+str(config.lr[0])+'_'+str(num_negatives)+'_'+str(epoch)
                torch.save(agree.state_dict(), current_weights_path)
                if best_checkpoint <= cur_checkpoint:
                    best_weights_path = current_weights_path
                    best_checkpoint = cur_checkpoint

NCF: embedding size 32, run Iteration: 100, #neg: 4
Iteration 0,	loss: [0.7709], time: [8.4s]
	 Evaluation done [39.4 s]
		 @5: HR = 0.0021, NDCG = 0.0008, Rec = 0.0021
		 @10: HR = 0.0021, NDCG = 0.0008, Rec = 0.0021
Iteration 1,	loss: [0.4308], time: [7.5s]
Iteration 2,	loss: [0.2598], time: [10.4s]
Iteration 3,	loss: [0.1991], time: [10.4s]
	 Evaluation done [39.2 s]
		 @5: HR = 0.0042, NDCG = 0.0017, Rec = 0.0042
		 @10: HR = 0.0064, NDCG = 0.0024, Rec = 0.0064
Iteration 4,	loss: [0.1717], time: [7.6s]
Iteration 5,	loss: [0.1576], time: [10.3s]
Iteration 6,	loss: [0.1498], time: [10.4s]
	 Evaluation done [31.2 s]
		 @5: HR = 0.0042, NDCG = 0.0017, Rec = 0.0042
		 @10: HR = 0.0106, NDCG = 0.0036, Rec = 0.0106
Iteration 7,	loss: [0.1442], time: [7.6s]
Iteration 8,	loss: [0.1415], time: [7.5s]
Iteration 9,	loss: [0.1398], time: [10.5s]
	 Evaluation done [27.0 s]
		 @5: HR = 0.0042, NDCG = 0.0032, Rec = 0.0042
		 @10: HR = 0.0148, NDCG = 0.0066, Rec = 0.0148
Iteration 10,	loss: [0.1388

In [8]:
print(best_weights_path)
agree = NCF(num_users, num_items, num_groups, config.embedding_size, gu_dict, config.drop_ratio)
agree.load_state_dict(torch.load(best_weights_path))
print('warm')
ret = evaluation(agree, test_warm, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
print('cold')
ret = evaluation(agree, test_cold, config.topK[:2], dataset.group_trainRatings, dataset.num_items)

weights/ncfdouban_32_0.0005_4_96
warm
	 Evaluation done [52.9 s]
		 @5: HR = 0.0266, NDCG = 0.0169, Rec = 0.0266
		 @10: HR = 0.0422, NDCG = 0.0220, Rec = 0.0422
cold
	 Evaluation done [185.7 s]
		 @5: HR = 0.0003, NDCG = 0.0001, Rec = 0.0003
		 @10: HR = 0.0008, NDCG = 0.0003, Rec = 0.0008


In [9]:

cold_groups = []
for g in test_cold.keys():
    cold_groups.append(g)
cold_group_indicies = torch.LongTensor(cold_groups)

In [10]:
g_shape = agree.groupembeds.groupEmbedding.weight[cold_group_indicies].shape



In [11]:
for _ in range(3):
    with torch.no_grad():
        agree.groupembeds.groupEmbedding.weight[cold_group_indicies] = torch.randn(g_shape)
    print('cold')
    ret = evaluation(agree, test_cold, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
    print('warm')
    ret = evaluation(agree, test_warm, config.topK[:2], dataset.group_trainRatings, dataset.num_items)   

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!